# Visualisation of Bessel-Gauss propagation

This notebook visualises the results of the Bessel-Gauss profiles intitiated by [this notebook](prepare_Bessel.ipynb). We select 4 of the results and animate the passage of the pulse through medium. See other notebooks ([link 1](../gas_cell/xxx), [link 2](../density_profile/analyse_density_profile.ipynb)) for different analyses.

## Load libraries & initial data

In [ ]:
## python modules used within this notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
from contextlib import ExitStack
import os
import h5py
import sys
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG
import plot_presets as pp 

matplotlib.rcParams['animation.embed_limit'] = 200.
%matplotlib inline

## Load data

Here we load the selected data from an input list. The provided test cases are for `30fs` and `60fs` (the durations of the initial pulses).

In [ ]:
demos_path = os.path.join(os.environ['MULTISCALE_DEMOS'],'Bessel') # path to the data, using the environment path to the demos
series_version = '30fs'                                            # '30fs' or '60fs'

h5files = [os.path.join(demos_path, series_version, foo) for foo in 
            ['results_Bessel_1.h5','results_Bessel_3.h5','results_Bessel_7.h5','results_Bessel_9.h5']] # list of loaded simulations

# # Mix the same results from different simulation series
# import itertools
# h5files = [os.path.join(demos_path, foo, bar) for foo, bar in 
#             itertools.product(['30fs','60fs'],['results_Bessel_3.h5','results_Bessel_9.h5'])]
             

# load the data
Nsim = len(h5files)
CUPRAD_res = []
with ExitStack() as stack:
    # Open all files and store file objects in a list
    files = [stack.enter_context(h5py.File(h5file, 'r')) for h5file in h5files]
    
    # Load data from each file and append it to CUPRAD_res
    for f in files:
        CUPRAD_res.append(dfC.get_data(f))
        CUPRAD_res[-1].vacuum_shift()
    for k1 in [1,3]:
        CUPRAD_res[k1].complexify_envel(output='add')                   # add complexified electric field profiles in the main class
    Intensity_map_Gauss  = mn.FieldToIntensitySI(abs(CUPRAD_res[1].E_zrt_cmplx_envel))
    Intensity_map_Bessel = mn.FieldToIntensitySI(abs(CUPRAD_res[3].E_zrt_cmplx_envel))


## Plot the propagating pulse
We plot the data for all the loaded simulations. We set the shared limits for the plots defined relatively to the pulse duration and set the stride (`frame_multiplier`) in $z$ for the plotting (using all the points might be resource consuming). We also choose whether to save the animation and set `fps` to ensure appealing output.

In [ ]:
tlim_relative_to_pulse_duration = (-1.75, 1.75)
tlim = [np.asarray([t_plot_span*result.pulse_duration_entry for t_plot_span in tlim_relative_to_pulse_duration]) for result in CUPRAD_res] # [fs]

frame_multiplier = 8 # stride in z for plotting

save_animation   = False
fps              = 5 # fps to synchronise output with the frame_multiplier
animation_author = 'Jan Vábek'



Finally, we generate the figure. it allows different shapes of input simulations. However, we assume a similar spacing in $z$, and the simulation animates until the shortest $z$-axis is consumed.

In [ ]:
# Code to generate the animated figure

# find subarrays (assume general case with different input archives)
k_t_min, k_t_max = tuple(zip(*[mn.FindInterval(1e15*CUPRAD_res[k1].tgrid,1.05*tlim[k1]) for k1 in range(len(CUPRAD_res))]))


Nrows = Nsim//2 if (Nsim%2==0) else (Nsim//2)+1

fig, axes = plt.subplots(Nrows, 2, figsize=(15, Nrows*4))  # Create a 2x2 grid of subplots
axes = axes.flatten()


pcs = []; cbars = []
# Create pcolormesh plots and colorbars
for k1, ax in enumerate(axes[:len(CUPRAD_res)]):
    pc = ax.pcolormesh(1e15*CUPRAD_res[k1].tgrid[k_t_min[k1]:k_t_max[k1]],
                       1e6*CUPRAD_res[k1].rgrid,
                       1e-9*CUPRAD_res[k1].E_zrt[0, :, k_t_min[k1]:k_t_max[k1]],
                       shading='auto', cmap='seismic')
    pcs.append(pc)
    cbar = fig.colorbar(pc, ax=ax)
    cbar.ax.set_ylabel(r'$\mathcal{E}$ [GV/m]', rotation=90)
    cbars.append(cbar)
    

    # Set axis properties
    ax.set_xlim(tlim[k1])
    ax.set_title("z={:.2f} mm".format(1e3*CUPRAD_res[k1].zgrid[0]))
    ax.set_xlabel(r'$t~[\mathrm{fs}]$')

for k1 in range(Nrows): axes[2*k1].set_ylabel(r'$\rho~[\mu\mathrm{m}]$')  # Set ylabel only for the left plots
for k1 in range(Nsim,2*Nrows): axes[k1].axis('off')

# Create colorbars using list comprehensions


def update(frame):
    for k1 in range(len(CUPRAD_res)):
        # Update the data for each subplot
        data = 1e-9*CUPRAD_res[k1].E_zrt[frame_multiplier*frame, :, k_t_min[k1]:k_t_max[k1]]
        
        # Update the colors
        pcs[k1].set_array(data.ravel())
        max_value_symmetric = np.max(np.abs((data.min(), data.max())))
        pcs[k1].set_clim(-max_value_symmetric,max_value_symmetric)
        cbars[k1].update_normal(pcs[k1])

        axes[k1].set_title("z={:.2f} mm".format(1e3*CUPRAD_res[k1].zgrid[frame_multiplier*frame]))

    return pcs

# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout() 

ani = matplotlib.animation.FuncAnimation(fig,
                                         update,
                                         frames=np.min([len(result.zgrid) for result in CUPRAD_res])//frame_multiplier,
                                         blit=True)

if save_animation:
    # Define the writer using ffmpeg for mp4 format and save it
    ani_outpath = os.path.join(os.environ['MULTISCALE_WORK_DIR'],'Bessel', 'export')
    if not(os.path.exists(ani_outpath)): os.makedirs(ani_outpath)

    FFmpegWriter = matplotlib.animation.writers['ffmpeg']
    writer = FFmpegWriter(fps=fps, metadata=dict(artist=animation_author), bitrate=1800)

    ani.save(os.path.join(ani_outpath,'Bessel_pulse_propagation.mp4'), writer=writer)

plt.close(fig)
HTML(ani.to_jshtml())


## Plot start end, max Gaussian, max Bessel-Gauss

In [ ]:
# k_z2 = mn.FindInterval(zgrid_Hankel ,z2)

# Hrange = (20,50)
# theta_range = (-5,5)
# orders_to_plot = 3          # the range of the logarithmic plot of the spatially resolved harmonic spectra

r_range = (-250,250) # microns
z_range = (0,8) # cm

fig, axs = plt.subplots(2, 2, figsize=(12, 6),
                        sharey='row',
                        gridspec_kw={'wspace': 0.05, 'hspace': 0.25})

# fig, axs = plt.subplots(1, 2, figsize=(12, 3)) #,
#                         # sharey='row',
#                         # gridspec_kw={'wspace': 0.05, 'hspace': 0.25})


k1 = 3
r_grid, sym_data = mn.symmetrize_y(
    1e6*CUPRAD_res[k1].rgrid,
    (
        HHG.ComputeCutoff(
            mn.FieldToIntensitySI(CUPRAD_res[k1].E_zrt[0, :, k_t_min[k1]:k_t_max[k1]]) / units.INTENSITYau,
            mn.ConvertPhoton(CUPRAD_res[k1].omega0, 'omegaSI', 'omegaau'),
            mn.ConvertPhoton(CUPRAD_res[k1].Ip_eV, 'eV', 'omegaau')
        )[1]
    ).T
)


pc1 = axs[0, 0].pcolormesh(1e15 * CUPRAD_res[k1].tgrid[k_t_min[k1]:k_t_max[k1]], r_grid, sym_data.T, shading='auto')

axs[0, 0].set_ylim(r_range)

cbar = fig.colorbar(pc1, ax=axs[0,0])
cbar.ax.set_ylabel(r'$\mathcal{E}$ [GV/m]', rotation=90)
cbars.append(cbar)


r_grid, sym_data = mn.symmetrize_y(
    1e6*CUPRAD_res[k1].rgrid,
    (
        HHG.ComputeCutoff(
            mn.FieldToIntensitySI(CUPRAD_res[k1].E_zrt[-1, :, k_t_min[k1]:k_t_max[k1]]) / units.INTENSITYau,
            mn.ConvertPhoton(CUPRAD_res[k1].omega0, 'omegaSI', 'omegaau'),
            mn.ConvertPhoton(CUPRAD_res[k1].Ip_eV, 'eV', 'omegaau')
        )[1]
    ).T
)


pc2 = axs[0, 1].pcolormesh(1e15 * CUPRAD_res[k1].tgrid[k_t_min[k1]:k_t_max[k1]], r_grid, sym_data.T, shading='auto')

axs[0, 1].set_ylim(r_range)

cbar = fig.colorbar(pc2, ax=axs[0,1])
cbar.ax.set_ylabel(r'$\mathcal{E}$ [GV/m]', rotation=90)
cbars.append(cbar)



# CUPRAD_res[k1].complexify_envel(output='add')                   # add complexified electric field profiles in the main class
# Intensity_map_Bessel = mn.FieldToIntensitySI(abs(CUPRAD_res[k1].E_zrt_cmplx_envel))

r_grid, sym_data = mn.symmetrize_y(
    1e6*CUPRAD_res[k1].rgrid,
    (
        HHG.ComputeCutoff(
            np.max(Intensity_map_Bessel,axis=2) / units.INTENSITYau,
            mn.ConvertPhoton(CUPRAD_res[k1].omega0, 'omegaSI', 'omegaau'),
            mn.ConvertPhoton(CUPRAD_res[k1].Ip_eV, 'eV', 'omegaau')
        )[1]
    )
)

pc3 = axs[1, 0].pcolormesh(1e3 * CUPRAD_res[k1].zgrid, r_grid, sym_data.T, shading='auto')

axs[1, 0].set_ylim(r_range)

cbar = fig.colorbar(pc3, ax=axs[1, 0])
cbar.ax.set_ylabel(r'$\mathcal{E}$ [GV/m]', rotation=90)
cbars.append(cbar)





k1=1
r_grid, sym_data = mn.symmetrize_y(
    1e6*CUPRAD_res[k1].rgrid,
    (
        HHG.ComputeCutoff(
            np.max(Intensity_map_Gauss,axis=2) / units.INTENSITYau,
            mn.ConvertPhoton(CUPRAD_res[k1].omega0, 'omegaSI', 'omegaau'),
            mn.ConvertPhoton(CUPRAD_res[k1].Ip_eV, 'eV', 'omegaau')
        )[1]
    )
)

pc4 = axs[1, 1].pcolormesh(1e3 * CUPRAD_res[k1].zgrid, r_grid, sym_data.T, shading='auto')

axs[1, 1].set_ylim(r_range)

cbar = fig.colorbar(pc4, ax=axs[1, 1])
cbar.ax.set_ylabel(r'$\mathcal{E}$ [GV/m]', rotation=90)
cbars.append(cbar)













# pc1 = axs[0].pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto')
# pc1 = axs[0,0].pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',norm=colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max()))
# cbar1 = fig.colorbar(pc1, ax=axs[0,0], orientation='vertical') # fraction=0.02, pad=0.02)
# cbar1.set_label(r'XUV signal [arb. u.]', rotation=90)










# sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(exit_plane_transform).T/data_max)[1]
# pc2 = axs[0,1].pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',norm=colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max()))
# cbar2 = fig.colorbar(pc2, ax=axs[0,1], orientation='vertical') # fraction=0.02, pad=0.02)
# cbar2.set_label(r'XUV signal [arb. u.]', rotation=90)

# sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(cumulative_field[-1,:,:]).T)[1]
# pc3 = axs[1,0].pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',norm=colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max()))
# cbar3 = fig.colorbar(pc3, ax=axs[1,0], orientation='vertical') # fraction=0.02, pad=0.02)
# cbar3.set_label(r'XUV signal [arb. u.]', rotation=90)

# # sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(cumulative_field[-1,:,:]).T)[1]
# pc4 = axs[1,1].pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto')
# cbar4 = fig.colorbar(pc4, ax=axs[1,1], orientation='vertical') # fraction=0.02, pad=0.02)
# cbar4.set_label(r'XUV signal [arb. u.]', rotation=90)

# for ax in [axs[1,0],axs[1,1]]:
#     ax.set_xlim(Hrange)
#     ax.set_xlabel(r'Harmonic order [-]')
    
# for ax in [axs[0,0],axs[1,0]]:
#     ax.set_ylim(theta_range)
#     ax.set_ylabel(r'Divergence [mrad]')

# # H_min, H_max = axs[0].get_xlim() 
# # odd_ticks_major = range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 10) # Generate odd ticks within the H-range
# # odd_ticks_minor = range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 2)  # Generate odd ticks within the H-range
# # for ax in axs:
# #     ax.set_xlabel('harmonic order [-]')
# #     ax.set_xticks(odd_ticks_major)
# #     ax.set_xticks(odd_ticks_minor, minor = True)

# corner_x, corner_y = 0.98, 0.95  # upper-right in Axes-fraction coords (0–1)

# axs[0, 0].text(corner_x, corner_y, 'a)', transform=axs[0, 0].transAxes, fontsize=16, va='top', ha='right', color='white')
# axs[0, 1].text(corner_x, corner_y, 'b)', transform=axs[0, 1].transAxes, fontsize=16, va='top', ha='right', color='white')
# axs[1, 0].text(corner_x, corner_y, 'c)', transform=axs[1, 0].transAxes, fontsize=16, va='top', ha='right', color='white')
# axs[1, 1].text(corner_x, corner_y, 'd)', transform=axs[1, 1].transAxes, fontsize=16, va='top', ha='right', color='white')


plt.show()
